In [1]:
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform

# 파일 경로 설정
EXCEL_PATH = "/Users/popi/Desktop/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/popi/Desktop/wp/files/"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 시스템에 따른 폰트 경로 설정
print("시스템 폰트 검색 중...")
font_registered = False

# macOS의 경우
if platform.system() == 'Darwin':
    possible_fonts = [
        '/System/Library/Fonts/AppleSDGothicNeo.ttc',
        '/System/Library/Fonts/AppleGothic.ttf',
        '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
        '/Library/Fonts/AppleGothic.ttf',
        '/Library/Fonts/NanumGothic.ttf',
        '/System/Library/Fonts/STHeiti Light.ttc',
        '/System/Library/Fonts/STHeiti Medium.ttc'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                # TTF 폰트 등록
                print(f"폰트 발견: {font_path}")
                if font_path.endswith('.ttc'):
                    # TTC(TrueType Collection) 파일 처리
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
                else:
                    pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue
# Windows의 경우
elif platform.system() == 'Windows':
    possible_fonts = [
        'C:\\Windows\\Fonts\\malgun.ttf',
        'C:\\Windows\\Fonts\\gulim.ttc',
        'C:\\Windows\\Fonts\\batang.ttc',
        'C:\\Windows\\Fonts\\NanumGothic.ttf'
    ]
    
    for font_path in possible_fonts:
        if os.path.exists(font_path):
            try:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
                font_registered = True
                print(f"폰트 등록 성공: {font_path}")
                break
            except Exception as e:
                print(f"폰트 등록 실패({font_path}): {e}")
                continue

# 폰트 등록에 실패한 경우 경고
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 한글이 제대로 표시되지 않을 수 있습니다.")
    # 기본 폰트 설정
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14  # 줄 간격
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 각 행에 대해 PDF 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = str(row['body'])
        
        # 디버깅 정보 출력
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 파일명에 사용할 수 없는 문자 제거
        safe_title = re.sub(r'[\\/*?:"<>|]', '', title)
        
        # 출력 파일 경로
        output_path = os.path.join(OUTPUT_DIR, f"{safe_title}.pdf")
        
        # PDF 문서 생성
        doc = SimpleDocTemplate(
            output_path,
            pagesize=A4,
            rightMargin=72,  # 1 inch
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        
        # 내용을 담을 리스트
        story = []
        
        # 제목 추가 (더 크고 굵게)
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        # HTML 예약 문자 처리
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # 본문 내용에서 줄바꿈 처리
        paragraphs = body.split('\n')
        
        for para in paragraphs:
            if para.strip() == '':
                # 빈 줄은 간격으로 처리
                story.append(Spacer(1, 0.1 * inch))
            else:
                # 문단 텍스트에 HTML 태그가 포함된 경우 처리
                para = para.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                
                # 여러 공백을 하나로 처리 (HTML에서 여러 공백은 하나로 취급됨)
                para = re.sub(r' +', ' ', para)
                
                # 문단 추가 
                story.append(Paragraph(para, styles['Normal_Justified']))
        
        # PDF 생성
        try:
            doc.build(story)
            print(f"PDF 생성 완료: {output_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
            import traceback
            traceback.print_exc()
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print("\n모든 PDF 변환 작업이 완료되었습니다.")

PermissionError: [Errno 13] Permission denied: '/Users/popi'